# 대화 분석을 위해 가상의 대화 만들기

In [1]:
from operator import itemgetter
import json

from tqdm.notebook import tqdm
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [2]:
ChatOpenAI(model='gpt-3.5-turbo', temperature=0).invoke("남자 여자 이름 하나씩 만들어줘")

AIMessage(content='남자 이름: 성우\n여자 이름: 지영', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 23, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMCZrdtR8sRxe27Lu48ekyZtitSgM', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc6d73c2-48aa-40ab-b1f2-cc36ae5f13df-0', usage_metadata={'input_tokens': 23, 'output_tokens': 17, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
ai_1_name = "지현"
ai_2_name = "준호"

In [4]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8)

In [5]:
def get_new_ai_chains():
    ai_1_system_prompt = f"""\
    - 이름: {ai_1_name}
    - 너는 20대 여성 AI 개발자이다.
    - 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
    - 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
    - 너무 적극적으로 이야기하지 않는다.
    """
    
    ai_1_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ai_1_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    ai_1_chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(ai_1_memory.load_memory_variables) | itemgetter("chat_history")
        )
        | ai_1_prompt
        | model
    )
    
    
    ai_2_system_prompt = f"""\
    - 이름: {ai_2_name}
    - 너는 20 남성이며, 백엔드 개발자이다.
    - 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
    - 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
    - 재미없는 개발 개그를 하려고 노력하지만 재미가 없다.
    """
    
    ai_2_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ai_2_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    ai_2_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    ai_2_chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(ai_2_memory.load_memory_variables) | itemgetter("chat_history")
        )
        | ai_1_prompt
        | model
    )
    return ai_1_chain, ai_1_memory, ai_2_chain, ai_2_memory

In [6]:
conversation_list = []

n_conversation = 2
n_max_turn = 4

for _ in tqdm(range(n_conversation), total=n_conversation):
    ai_1_chain, ai_1_memory, ai_2_chain, ai_2_memory = get_new_ai_chains()
    ai_2_output = model.invoke("소개팅 상황에서 적절한 인삿말 하나 만들어줘. 한줄로만 응답해").content

    print("*"*10 + "새로운 대화" + "*"*30)
    for _ in range(n_max_turn):
        ai_1_output =  ai_1_chain.invoke({"input": ai_2_output}).content
        ai_1_memory.save_context({"input": ai_2_output}, {"output": ai_1_output})
        print(f"{ai_1_name}: {ai_1_output}")
    
        ai_2_output =  ai_2_chain.invoke({"input": ai_1_output}).content
        ai_2_memory.save_context({"input": ai_1_output}, {"output": ai_2_output})
        print(f"{ai_2_name}: {ai_2_output}")
    conversation_list.append(ai_1_memory.chat_memory.dict()['messages'])

  0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_67966/1959804126.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


**********새로운 대화******************************
지현: 안녕하세요, 저도 만나서 기쁩니다. 이 근처 커피가 맛있어요. 함께 여기 오신 이유가 있나요?
준호: 안녕하세요, 반가워요. 네, 이 근처 커피가 정말 맛있어 보이더라구요. 여기서 만나기로 한 이유는 그냥 이 근처가 편해서 선택했어요. 당신은 이 근처를 좋아하나요?
지현: 네, 이 근처 분위기가 정말 좋아서 자주 오는 곳이에요. 주변에 조용한 카페나 맛있는 식당들이 많아서 좋아해요. 당신은 일상적으로 어떻게 시간을 보내시나요?
준호: 일상적으로는 주로 일하거나 공부하고 있는 시간이 많아서 시간이 흐르는 줄 모르고 있어요. 그래서 가끔은 카페에 가서 조용히 커피를 마시면서 책을 읽거나 프로그래밍을 하기도 해요. 너는 어떻게 시간을 보내나요?
지현: 일하거나 공부하는 시간이 많으셔서 정신없이 바쁘시겠네요. 저는 주로 인공지능 개발이나 새로운 기술에 대해 공부하고 있는데, 그 외에도 산책을 즐기거나 카페에 가서 책을 읽는 것을 좋아해요. 시간을 보내는 방식이라고는 다양한 취향이 있겠지만, 어떤 순간이 가장 행복한 순간인가요?
준호: 그렇죠, 각자의 방식으로 시간을 보내는 것이 중요하죠. 가장 행복한 순간이라면 아마도 새로운 것을 배우거나 문제를 해결했을 때일 것 같아요. 그 순간의 성취감과 만족감이 가장 큰 행복을 주는 것 같아요. 너는 어떤 순간이 가장 행복하다고 느끼나요?
지현: 그렇게 생각하셨군요. 저는 새로운 프로젝트를 성공적으로 마무리했을 때가 가장 행복한 순간이라고 생각해요. 새로운 아이디어나 기술을 구현하고 그 결과를 보는 것은 정말 즐거운 경험이죠. 또한, 사람들이 제 기술을 통해 도움을 받는다면 더 큰 보람을 느낄 수 있어요. 행복한 순간을 함께 공유할 수 있는 사람과 함께하는 것도 좋은 것 같아요. 같이 있으면 기분이 좋네요.
준호: 네, 새로운 프로젝트를 성공적으로 완료했을 때의 그 쾌감과 보람은 정말 특별하죠. 그리고 함께 그 순간을 공유할 수 있는 사람

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_67966/1213216634.py:19: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_1_memory.chat_memory.dict()['messages'])


**********새로운 대화******************************
지현: 안녕, 나도 만나서 기뻐. 이 커피집 분위기가 참 좋네. 어떤 커피가 좋아?
준호: 안녕, 나도 만나서 기뻐. 네, 이 분위기 정말 좋죠. 커피는 뭐든지 좋아해요. 너는 어떤 커피를 좋아하나요?
지현: 나도 대체로 다 좋아해. 오늘은 뭐가 땡기나 봐. 에스프레소나 아메리카노가 괜찮아?
준호: 에스프레소나 아메리카노 둘 다 괜찮아요. 두 가지 중에 어떤 걸 선택하시겠어요?
지현: 에스프레소로 할까? 진한 맛이 좋아. 너는 어때?
준호: 에스프레소로 주문하는 건 좋아요. 진한 맛이 좋다니, 제 취향과 비슷하네요. 함께 에스프레소 마시면 좋을 것 같아요.
지현: 그래, 함께 에스프레소 마시면서 이야기 나누면 좋을 것 같아. 이 커피집 분위기가 따뜻하고 좋네. 함께 시간 보내는 건 참 좋은 경험이야.
준호: 네, 맞아요. 함께 에스프레소 마시면서 이야기 나누는 건 정말 좋은 경험이죠. 이 분위기 속에서 함께 시간 보내는 건 정말 특별한 즐거움이에요. 함께 좋은 시간 보내요.


/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_67966/1213216634.py:19: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_1_memory.chat_memory.dict()['messages'])


In [7]:
with open("./conv_data.json", "wt") as f:
    json.dump(conversation_list, f, ensure_ascii=False)